# News Recommendation Prototype

This notebook demonstrates a quick prototype of generating recommendations using the trained recommender model.

In [1]:
import torch
import pandas as pd
from pathlib import Path

from src.recommender.model import MFModel

In [2]:
# Load processed data
news = pd.read_csv('../data/processed/processed_news.csv')
unique_users = news['user_id'].unique()
unique_items = news['article_id'].unique()
user_to_idx = {u:i for i,u in enumerate(unique_users)}
idx_to_user = {i:u for u,i in user_to_idx.items()}
item_to_idx = {i:idx for idx,i in enumerate(unique_items)}
idx_to_item = {idx:i for i,idx in item_to_idx.items()}


## Load Trained Model

In [3]:
model_path = Path('../recommender_model.pth')
model = MFModel(num_users=len(unique_users), num_items=len(unique_items))
model.load_state_dict(torch.load(model_path))
model.eval()

/var/folders/q6/d41hkr0s6j1bbxl7n9yh223m0000gn/T/ipykernel_29845/2662941789.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))

MFModel(
  (user_embedding): Embedding(19, 64)
  (item_embedding): Embedding(41, 64)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)

## Generate Recommendations for a Sample User

In [4]:
sample_user = unique_users[0]
user_idx = user_to_idx[sample_user]
user_tensor = torch.tensor([user_idx]*len(unique_items), dtype=torch.long)
items_tensor = torch.tensor(range(len(unique_items)), dtype=torch.long)
with torch.no_grad():
    scores = model(user_tensor, items_tensor)

top_k = 5
recommended_indices = torch.argsort(scores, descending=True)[:top_k]
recommended_items = [idx_to_item[i.item()] for i in recommended_indices]
recommended_items

[np.int64(162), np.int64(117), np.int64(153), np.int64(180), np.int64(114)]

The recommended item IDs above represent the top articles for the given user. You can adjust the logic as needed, integrate metadata, or refine the model further.